In [1]:
import pandas as pd

In [4]:
senate_df = pd.read_csv('../source_data/1976-2020-senate.csv')
senate_df = senate_df[(senate_df['state_po'] != 'GA') | (senate_df['year'] != 2020)]
senate_df.loc[senate_df['year'] == 2021, 'year'] = 2020
senate_df = senate_df[~senate_df['writein']]
senate_df = senate_df[['year', 'state_po', 'candidate', 'party_simplified', 'candidatevotes']]
senate_df = senate_df.rename(columns = {'state_po' : 'state', 'party_simplified' : 'party', 'candidatevotes':'votes'})

senate_df.loc[(senate_df['candidate'] == 'BERNARD SANDERS') | (senate_df['candidate'] == 'BERNIE SANDERS'), 'party'] = 'DEMOCRAT'
senate_df.loc[(senate_df['candidate'].str.contains('KING') & senate_df['candidate'].str.contains('ANGUS')), 'party'] = 'DEMOCRAT'
senate_df.loc[senate_df['candidate'].str.contains('LIEBERMAN'), 'party'] = 'DEMOCRAT'

senate_df.loc[(senate_df['candidate'] == 'HARRY F. BYRD, JR.'), 'party'] = 'REPUBLICAN'

senate_df.loc[(senate_df['candidate'] == 'NANCY B. SPANNAUS'), ['party', 'votes']] = ['DEMOCRAT', 0]
senate_df.loc[(senate_df['candidate'] == 'JED WHITTAKER'), 'party'] = 'DEMOCRAT'
senate_df.loc[(senate_df['year'] == 2000) & (senate_df['state'] == 'AZ') & (senate_df['party'] == 'OTHER'), 'party'] = 'DEMOCRAT'

senate_df.loc[(senate_df['candidate'] == 'MACK F. MATTINGLY'), 'party'] = 'REPUBLICAN'

senate_df.loc[(senate_df['candidate'] == 'GEORGE COOK'), 'party'] = 'REPUBLICAN'
senate_df.loc[(senate_df['candidate'] == 'STEVEN A. ROSILE') & (senate_df['year'] == 2002), 'party'] = 'DEMOCRAT'

senate_df.loc[(senate_df['year'] == 2002) & (senate_df['state'] == 'MA') & (senate_df['party'] != 'DEMOCRAT'), 'votes'] = 0
senate_df.loc[(senate_df['year'] == 2002) & (senate_df['state'] == 'MA') & (senate_df['party'] != 'DEMOCRAT'), 'party'] = 'REPUBLICAN'

senate_df.loc[(senate_df['candidate'] == 'SHAWN O\'HARA') & (senate_df['year'] == 2002), ['party', 'votes']] = ['DEMOCRAT', 0]
senate_df.loc[(senate_df['candidate'] == 'STEVE OSBORN'), 'party'] = 'DEMOCRAT'

senate_df.loc[(senate_df['candidate'] == 'REBEKAH KENNEDY'), ['party', 'votes']] = ['REPUBLICAN', 0]

senate_df = senate_df[~((senate_df['candidate'] == 'CHARLIE CRIST') & (senate_df['year'] == 2010))]

senate_df.loc[(senate_df['candidate'] == 'GREG ORMAN'), 'party'] = 'DEMOCRAT'

senate_df.loc[(senate_df['candidate'] == 'JOE MILLER'), 'party'] = 'REPUBLICAN'

senate_df.loc[(senate_df['candidate'] == 'MARGARET STOCK'), 'party'] = 'DEMOCRAT'

senate_df.loc[(senate_df['candidate'] == 'RICKY DALE HARRINGTON JR.'), ['party', 'votes']] = ['DEMOCRAT', 0]

senate_df.loc[(senate_df['candidate'] == 'CYNTHIA M. LUMMIS'), 'party'] = 'REPUBLICAN'
senate_df.loc[(senate_df['candidate'] == 'MERAV BEN DAVID'), 'party'] = 'DEMOCRAT'

In [5]:
senate_df = senate_df.groupby(['year', 'state', 'party'])[['votes']]\
    .sum()\
    .reset_index()\
    .groupby(['year', 'state'])[['party', 'votes']]\
    .apply(lambda x : x[x['party'].str.match('DEMOCRAT|REPUBLICAN')])\
    .reset_index().drop(columns = 'level_2')

senate_df = senate_df.groupby(['year', 'state'])[['party', 'votes']]\
                .apply(lambda table: pd.Series({'dem' : table[table['party'] == 'DEMOCRAT'].reset_index()['votes'].get(0,0),
                                            'rep' : table[table['party'] == 'REPUBLICAN'].reset_index()['votes'].get(0,0)}))\
                .reset_index()

senate_df['dem_pct'] = senate_df['dem'].div(senate_df['dem'] + senate_df['rep'], axis = 0)
senate_df['rep_pct'] = senate_df['rep'].div(senate_df['dem'] + senate_df['rep'], axis = 0)

senate_df = senate_df[['year', 'state', 'dem_pct', 'rep_pct']]

senate_df.to_csv('../processed_data/1976-2020-senate.csv', header = True, index = False)